In [14]:
#!/usr/bin/env python2
# -*- coding=utf-8 -*-
from queue import Queue
from threading import Thread
import threading
import time

from lxml import etree
import requests

url_list = []
lock = threading.Lock()


class DouBanSpider(Thread):
    def __init__(self, q) :
        # 重寫寫父類的__init__方法
        super(DouBanSpider, self).__init__()
        self.q = q
        self.headers = {
            'Host': 'movie.douban.com',
            'Referer': 'https://movie.douban.com/top250?start=225&filter=',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.104 Safari/537.36',
        }

    def run(self):
        self.parse_page()

    def send_request(self, url):
        '''
        用來傳送請求的方法
        :return: 返回網頁原始碼
        '''
        # 請求出錯時，重複請求３次,
        i = 0
        while i <= 3:
            try:
                print
                u"[INFO]請求url:" + url
                html = requests.get(url=url, headers=self.headers).content
            except Exception as e:
                print
                u'[INFO] %s%s' % (e, url)
                i += 1
            else:
                return html

    def parse_page(self):
        '''
        解析網站原始碼，並採用ｘｐａｔｈ提取　電影名稱和平分放到佇列中
        :return:
        '''
        while 1:
            try:

                url = url_list.pop()
            except IndexError as e:
                break
            lock.acquire()
            response = self.send_request(url)
            html = etree.HTML(response)
            # 　獲取到一頁的電影資料
            node_list = html.xpath("//div[@class='info']")
            for move in node_list:
                # 電影名稱
                title = move.xpath('.//a/span/text()')[0]
                # 評分
                score = move.xpath('.//div[@class="bd"]//span[@class="rating_num"]/text()')[0]

                # 將每一部電影的名稱跟評分加入到佇列
                self.q.put(score + "\t" + title)
            
            lock.release()


class AllUrlSpider(Thread):
    def run(self):
        base_url = 'https://movie.douban.com/top250?start='
        # 構造所有ｕｒｌ
        for num in range(225, -1, -25):
            url_list.append(base_url + str(num))
            print("獲得URL：{}".format(base_url + str(num)))


def main():
    # 建立一個佇列用來儲存程式獲取到的資料
    q = Queue()
    a = AllUrlSpider()
    a.start()

    # 儲存執行緒
    Thread_list = []
    # 建立並啟動執行緒
    for i in range(5):
        p = DouBanSpider(q)
        p.start()
        Thread_list.append(p)

    a.join()
    # 讓主執行緒等待子執行緒執行完成
    for i in Thread_list:
        i.join()

    while not q.empty():
        print(q.get())


if __name__ == "__main__":
    start = time.time()
    main()
    print('[info]耗時：%s' % (time.time() - start))

獲得URL：https://movie.douban.com/top250?start=225
獲得URL：https://movie.douban.com/top250?start=200
獲得URL：https://movie.douban.com/top250?start=175
獲得URL：https://movie.douban.com/top250?start=150
獲得URL：https://movie.douban.com/top250?start=125
獲得URL：https://movie.douban.com/top250?start=100
獲得URL：https://movie.douban.com/top250?start=75
獲得URL：https://movie.douban.com/top250?start=50
獲得URL：https://movie.douban.com/top250?start=25
獲得URL：https://movie.douban.com/top250?start=0
9.1	海洋
8.5	彗星来的那一夜
8.7	谍影重重2
8.7	战争之王
8.6	穿越时空的少女
8.5	阿飞正传
8.8	千钧一发
8.9	地球上的星星
8.5	完美陌生人
8.6	黑客帝国2：重装上阵
8.6	谍影重重
9.2	东京物语
8.6	崖上的波妞
8.5	香水
9.1	我爱你
9.5	茶馆
8.6	再次出发之纽约遇见你
8.6	朗读者
8.8	聚焦
8.3	驴得水
8.7	浪潮
8.5	猜火车
8.4	爱乐之城
8.4	小萝莉的猴神大叔
9.5	背靠背，脸对脸
8.7	天使爱美丽
8.7	功夫
8.9	超脱
8.9	幽灵公主
9.0	小森林 夏秋篇
8.9	第六感
8.7	唐伯虎点秋香
8.8	重庆森林
8.8	阳光灿烂的日子
8.8	爱在黎明破晓前
9.5	人生果实
9.1	一一
8.8	菊次郎的夏天
8.8	蝙蝠侠：黑暗骑士崛起
8.8	哈利·波特与阿兹卡班的囚徒
9.0	小森林 冬春篇
8.7	消失的爱人
9.1	无人知晓
8.9	入殓师
8.7	超能陆战队
8.7	倩女幽魂
8.9	侧耳倾听
9.1	完美的世界
8.9	借东西的小人阿莉埃蒂
8.9	甜蜜蜜
8.9	爱在日落黄昏时
8.9	萤火之森
8.7	驯龙

In [37]:
# -*- coding=utf-8 -*-
from queue import Queue
from threading import Thread
import threading
import time

from lxml import etree
import requests

url_list = []
lock = threading.Lock()


class DouBanSpider(Thread):
    def __init__(self, q) :
        # 重寫寫父類的__init__方法
        super(DouBanSpider, self).__init__()
        self.q = q
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36',
        }

    def run(self):
        self.parse_page()

    def send_request(self, url):
        '''
        用來傳送請求的方法
        :return: 返回網頁原始碼
        '''
        # 請求出錯時，重複請求３次,
        i = 0
        while i <= 3:
            try:
                print
                u"[INFO]請求url:" + url
                html = requests.get(url=url, headers=self.headers).content
            except Exception as e:
                print
                u'[INFO] %s%s' % (e, url)
                i += 1
            else:
                return html

    def parse_page(self):
        '''
        解析網站原始碼，並採用ｘｐａｔｈ提取　電影名稱和平分放到佇列中
        :return:
        '''
        while 1:
            try:
                url = url_list.pop()
            except IndexError as e:
                break
            lock.acquire()
            response = self.send_request(url)
            html = etree.HTML(response)
             # 　獲取到一頁的電影資料
            
            # 電影名稱
            title = html.xpath('//div[@class="movie_intro_info_r"]/h1/text()')[0]
            # 評分
            score = html.xpath('//div[@class="movie_intro_info_r"]/span/text()')[3]
            
            # 將每一部電影的名稱跟評分加入到佇列
            self.q.put(str(score) + "\t" + str(title))
            lock.release()
            
class AllUrlSpider(Thread):
    def run(self):
        base_url = 'https://movies.yahoo.com.tw/movieinfo_main/%E9%BB%91%E5%AF%A1%E5%A9%A6-black-widow-10397'
        # 構造所有ｕｒｌ
        for num in range(0, 1):
            url_list.append(base_url)
            print("獲得URL：{}".format(base_url))

def main():
    # 建立一個佇列用來儲存程式獲取到的資料
    q = Queue()
    a = AllUrlSpider()
    a.start()

    # 儲存執行緒
    Thread_list = []
    # 建立並啟動執行緒
    for i in range(5):
        p = DouBanSpider(q)
        p.start()
        Thread_list.append(p)

    a.join()
    # 讓主執行緒等待子執行緒執行完成
    for i in Thread_list:
        i.join()

    while not q.empty():
        print(q.get())


if __name__ == "__main__":
    start = time.time()
    main()
    print('[info]耗時：%s' % (time.time() - start))
    

獲得URL：https://movies.yahoo.com.tw/movieinfo_main/%E9%BB%91%E5%AF%A1%E5%A9%A6-black-widow-10397


Exception in thread Thread-180:
Traceback (most recent call last):
  File "/usr/local/opt/python@3.8/Frameworks/Python.framework/Versions/3.8/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/var/folders/7t/bb_qyqxs0s1d1slg3618dg8r0000gn/T/ipykernel_69740/2419529837.py", line 24, in run
  File "/var/folders/7t/bb_qyqxs0s1d1slg3618dg8r0000gn/T/ipykernel_69740/2419529837.py", line 64, in parse_page
NameError: name 'new' is not defined


[info]耗時：0.5920240879058838
